In [12]:
# ---------------------
# IMPORTS
# ---------------------
import hashlib
from datetime import datetime
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import serialization

# ---------------------
# BLOCK CLASS
# ---------------------
class Block:
    # Initialization of the block with its attributes
    def __init__(self, index, timestamp, data, previous_hash, nonce=0):
        self.index = index
        self.timestamp = timestamp
        self.data = data
        self.previous_hash = previous_hash
        self.nonce = nonce
        self.hash = self.hash_block()

    # Hashing the block's attributes to generate its unique hash
    def hash_block(self):
        sha = hashlib.sha256()
        sha.update(str(self.index).encode('utf-8') +
                   str(self.timestamp).encode('utf-8') +
                   str(self.data).encode('utf-8') +
                   str(self.previous_hash).encode('utf-8') +
                   str(self.nonce).encode('utf-8'))
        return sha.hexdigest()

    # Mining the block by trying different nonce values until the hash meets the difficulty criteria
    def mine_block(self, difficulty):
        while self.hash[:difficulty] != "0" * difficulty:
            self.nonce += 1
            self.hash = self.hash_block()

# ---------------------
# GENESIS BLOCK CREATION
# ---------------------
def make_genesis_block():
    """Make the first block in a block-chain."""
    block = Block(index=0,
                  timestamp=datetime.now(),
                  data="Genesis Block",
                  previous_hash="0")
    return block

# ---------------------
# KEY PAIR GENERATION
# ---------------------
def generate_key_pair():
    private_key = ec.generate_private_key(ec.SECP256R1())
    public_key = private_key.public_key()

    pem = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    with open("public_key.pem", "wb") as f:
        f.write(pem)

    return private_key, public_key

# ---------------------
# DIGITAL SIGNATURE
# ---------------------
def sign_data(private_key, data):
    signature = private_key.sign(data.encode('utf-8'), ec.ECDSA(hashes.SHA256()))
    return signature

# Convert the public key object to PEM format string
def serialize_public_key(public_key):
    return public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    ).decode('utf-8')

# Verify the digital signature using the public key
def verify_signature(public_key, data, signature):
    try:
        public_key.verify(signature, data.encode('utf-8'), ec.ECDSA(hashes.SHA256()))
        return True
    except:
        return False

# ---------------------
# PUBLIC KEY STORAGE
# ---------------------
public_keys_storage = []

def store_public_key(public_key):
    serialized_key = serialize_public_key(public_key)
    public_keys_storage.append(serialized_key)    

# ---------------------
# BLOCK CREATION
# ---------------------
def next_block(pre_block, data='', difficulty=5):
    new_block = Block(index=pre_block.index + 1,
                      timestamp=datetime.now(),
                      data=data,
                      previous_hash=pre_block.hash)
    new_block.mine_block(difficulty=difficulty)
    return new_block

# Create a series of blocks for the blockchain
def create_blocks(difficulty=5):
    blockchain = [make_genesis_block()]
    for i in range(1, 21):
        data = f"Block #{i}"
        new_block = next_block(blockchain[-1], data, difficulty)
        blockchain.append(new_block)
    return blockchain




def convert_keys_to_strings(private_key, public_key):
    # Convert private key to PEM format
    private_pem = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    )

    # Convert public key to PEM format
    public_pem = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )

    # Convert bytes to strings
    private_key_str = private_pem.decode('utf-8')
    public_key_str = public_pem.decode('utf-8')

    return private_key_str, public_key_str

def dedo():
     private_key, public_key = generate_key_pair()
     return convert_keys_to_strings(private_key, public_key)


def sign():

    
# ---------------------
# MAIN EXECUTION
# ---------------------
if __name__ == "__main__":
    # Generate key pair
    private_key, public_key = generate_key_pair()
    store_public_key(public_key)
    
    # Sign and verify a sample data
    data = "Test Data"
    signature = sign_data(private_key, data)
    print("Public Key:\n", serialize_public_key(public_key))
    print("Signature:", signature)
    print("Verification:", verify_signature(public_key, data, signature))
    
    # Create the blockchain with 20 blocks
    blockchain = create_blocks(difficulty=5)

    # Print the blocks
    for block in blockchain:
        print(f"Block #{block.index} - Data: {block.data} - Hash: {block.hash}")

    # Print stored public keys
    print("\nStored Public Keys:")
    for key in public_keys_storage:
        print(key)


Public Key:
 -----BEGIN PUBLIC KEY-----
MFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAEdWUS1fBbeZROXWXeS2h4JuNiXj8q
8fb1USe2hd2yTmI08J5iyxj4iyz5+mXFfWVa78gWVJTfB6sdUMiHG/ToQA==
-----END PUBLIC KEY-----

Signature: b'0E\x02 @\xb8\xa5\x0f\xde\x01\xfa\xdfV\x10\x1e\x94e{\xfeMtA\x8f\x1f\xba\xef\x83\xcc\xee\xce\xdc9i\xca\xf1:\x02!\x00\xe5\xc0\x8c\xf5\xdf\r\x12\xbc\xb9}\xd0\xc2\x92\x1d\x8f\xe2\xad\xeb\x87\xbc\x02\xe1t\x9c\xab\x86\x9b\x10\x06\xf7<3'
Verification: True
Block #0 - Data: Genesis Block - Hash: ba83cc6b986ca0dad2dde400c2de5cb05348b7d2b218b5b06516699af9dee987
Block #1 - Data: Block #1 - Hash: 00000bc49b4d3cf47bd214c83f107fbc96b362fb3b350b34e32784377ea0724b
Block #2 - Data: Block #2 - Hash: 0000066c445f4c06e40ed4919f1377289358f143e4db06afc612cd94b4cadae9
Block #3 - Data: Block #3 - Hash: 00000e33987aa4cab7247100ba77fb46c2e21b06a2187e4d96cec252a52d5bba
Block #4 - Data: Block #4 - Hash: 00000274fd3e3ac34ca3d2641bdae90b6fb4c471569e6d22d63faffc387a2c28
Block #5 - Data: Block #5 - Hash: 00000cdc2d0d